In [ ]:
import sqlite3

from prettytable import PrettyTable

The [sqlite3](https://docs.python.org/2/library/sqlite3.html) library has been available as part of the Python Standard Library as of version 2.5, and _is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language._

We'll also be using the [prettytable](http://code.google.com/p/prettytable/wiki/Tutorial) library to print out the results nicely. 

It was already present in my Linux Mint version of PyCharm, but if needed, [installation instructions should be here.](http://code.google.com/p/prettytable/wiki/Installation)

---

Ok, so let's get started! 

First create a [Connection](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection) object that represents the database.

--- 

Noticing that the only parameters that are changing are the database name and query, we can just wrap it in a quick function like any good ~~lazy~~ programmer would.

In [ ]:
def query_db(database_name, query):
    '''Opens a connections to a SQL database using the sql3lite Python library,
        and runs a provided query on the database.

    Args:
        database_name (string): String containing the name of the database to
         be opened.

        query (string): SQL query to be executed on the database

    Returns:
        tabled_results (PrettyTable object): Results in a table to be printed.
    '''
    connection = sqlite3.connect(database_name)
    cursor = connection.cursor()
    results_generator = cursor.execute(query)

    tabled_results = pprint_results(results_generator)
    return tabled_results
    
def pprint_results(results_generator):
    ''' Formats results from sql3lite query into a nicer format to be printed.

    Add columns from the sql database, and ascii separators.

    Note: Put all results into memory to construct table.

    Args:
        results_generator (sqlite3.Cursor object): Cursor object containing the
            results after executing a query on it.

    Returns:
        table_output (PrettyTable object): A PrettyTable object containing the
         results of the SQL query in a nicely formatted container.
    '''

    colnames = [colname[0] for colname in results_generator.description]

    table_output = PrettyTable(colnames)
    table_output.padding_width = 1

    for row in results_generator:
         table_output.add_row(row)

    return table_output

In [ ]:
database_name = 'reuters.db'

In [ ]:
query = '''
SELECT *
FROM frequency
WHERE term = 'net'
AND count = 5
'''

In [ ]:
print query_db(database_name, query)

In [ ]:
new_query = '''
SELECT *
FROM frequency
WHERE term = 'net'
AND count = 6
'''

In [ ]:
print query_db(database_name, new_query)

Quick aside regarding the instructions:



---
_Many questions ask you to count the number of records returned by a query. Perhaps the easiest way to count the number of records returned by a query Q is to write Q as a subquery:_

```SELECT count(*) FROM (
  SELECT ...
) x;```

_(In SQLite, the alias "x" is not required, but in other dialects of SQL it is. So we've included it here.)_

---

If you're like me, and at first said.. "what?", hopefully this helps!

#### Nesting SQL queries working example

In [ ]:
connection = sqlite3.connect('reuters.db')

Once we have a a [Connection](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection), you can create a [Cursor](https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor) object which will execute SQL statements/queries on the database with its [.execute()](https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.execute) method.

In [ ]:
cursor = connection.cursor()

In [ ]:
query = '''
SELECT *
FROM frequency as f
WHERE term = 'net'
AND count = 5
'''

In [ ]:
results_generator = cursor.execute(query)

Executing our query with the Cursor object returns a [generator](https://www.jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) which yields each new result without smashing everything into memory at once.

It has a [.next()](http://anandology.com/python-practice-book/iterators.html) method if we want to the next result. We can run this until all results are exhausted, or just iterate through with a for loop.

In [ ]:
results_generator.next()

Let's execute our query again real quick, since we 'yielded the first result out of our object' in our previous example.

## [(Pretty) Printing](https://code.google.com/p/prettytable/wiki/Tutorial) out our results table.

Column names are stored in the description attribute of the sqlite3 cursor object.

It is a list of tuples with the first position containing the column name.